In [1]:
import re
import glob


import skipthoughts
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import TSNE

In [2]:
model = skipthoughts.load_model()
encoder = skipthoughts.Encoder(model)

/home/arunhiremath/sfsu/nlt/text_summarization/skipthoughts/Text-Summarization/data/ /home/arunhiremath/sfsu/nlt/text_summarization/skipthoughts/Text-Summarization/data/uni_skip.npz
Loading model parameters...
<built-in method items of collections.OrderedDict object at 0x7facda224040>
<built-in method items of collections.OrderedDict object at 0x7facda192440>
Compiling encoders...
Loading tables...
/home/arunhiremath/sfsu/nlt/text_summarization/skipthoughts/Text-Summarization/data/
Packing up...


In [3]:
def load_encoder_dictionary():
    encoder_word_file = open("./data/dictionary.txt", "rt")
    encoder_words = encoder_word_file.readlines()
    return encoder_words
    
    

In [4]:
debug = True
def setDebugFlag(isDebug):
    debug = isDebug



In [5]:
def sentence_spliter(text):
    #split the sentence based on the occurance of the [. or ! or ?]
    #split the sentence based on the occurance of ] or ) or " or 
    #eturn re.split(' *[\.\?\!][\'"\)\]]* *', text)
    sent_tokenize_list = sent_tokenize(text)
    return sent_tokenize_list
    

In [6]:
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [7]:
def strip_numbers(s):
    return ''.join(c for c in s if not c.isdigit())

In [8]:
contractions_dict = {     
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "iit will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [9]:
def expand_contractions(text, contractions_dict):
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match) \
            if contractions_dict.get(match) \
            else contractions_dict.get(match.lower())
        expanded_contraction = expanded_contraction
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [10]:
use_preprocessor = False

In [11]:
def sentence_preprocessor(sentence_list, encoder_words):
    filterd_sentences = []
    for sentence in sentence_list:
        corrected_sentence = expand_contractions(sentence, contractions_dict)
        corrected_sentence = strip_punctuation(sentence)
        corrected_sentence = strip_numbers(corrected_sentence)
        filterd_sentences.append(corrected_sentence)   
    return filterd_sentences
    


In [12]:
#This is going to be dataframe for our processing

#| key | file-path-to-original-text | gt-1 | gt-2 | gt-3 | gt-4 | gt-5 | predited-1 | predicted-2 |

In [13]:
inputFiles = "/home/arunhiremath/sfsu/nlt/text_summarization/skipthoughts/Text-Summarization/data-set/opinosis-dataset"
goldSummaryFiles = "/home/arunhiremath/sfsu/nlt/text_summarization/skipthoughts/Text-Summarization/data-set/gold-summary"
opinosis_train_test_dict_files = {}


In [14]:
#Get a list of all files from the opinosis data-set
from os import walk
from os.path import join
import os
import ntpath
def get_file_list(path):
    fileNames = []
    for (dirpath, dirnames, file_names) in walk(path):
        fileNames.extend(file_names)
        break;
    fileLists = []
    for fName in fileNames:
        fileLists.append(join(path,fName))
    return fileLists

In [15]:
def get_directory_list(path):
    directoryNames = []
    for (dirpath, dirnames, filenames) in walk(path):
        directoryNames.extend(dirnames)
    directoryList = []
    for dirNames in directoryNames:
        directoryList.append(join(path,dirNames))
    return directoryList

In [16]:
test_data_set_files = get_file_list(inputFiles)
gold_summary_folders = get_directory_list(goldSummaryFiles)


In [17]:
#This is going to be dataframe for our processing

#file-path-to-original-text | gt-1 | gt-2 | gt-3 | gt-4 | gt-5 | predited-1 | predicted-2 |

In [18]:
column_=["key","location","gt-1", "gt-2", "gt-3", "gt-4", "gt-5", "pd1", "pd2"]
allDataFrames = []
for test_file in test_data_set_files:
    data_frame = ["None","None","None","None","None","None","None", "None", "None"]
    data_frame[0]=os.path.splitext(os.path.basename(test_file))[0]
    fName = os.path.splitext(os.path.basename(test_file))[0]
    fName = fName.split(".")
    dName = join(goldSummaryFiles,fName[0])
    data_frame[1]=test_file
    summary_files = get_file_list(dName)
    temp = 2
    for file in summary_files:
        gold_summary = open(file, "rt")
        gold_summary_text = gold_summary.readlines()
        data_frame[temp]=gold_summary_text
        temp = temp + 1
    allDataFrames.append(data_frame)                    

In [19]:
import pandas as pd
dfObj = pd.DataFrame(allDataFrames, columns = column_)
dfObj.head(51)

,key,location,gt-1,gt-2,gt-3,gt-4,gt-5,pd1,pd2
0,eyesight-issues_amazon_kindle.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,[Its very soothing for the eyes.],"[Most users do not encounter eyestrain.\n, The...","[Most users do not encounter eyestrain.\n, The...","[E-ink makes it very easy on the eyes.\n, Allo...","[Very easy on the eyes.\n, No fatigue or strai...",None,None
1,food_holiday_inn_london.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,[The food in the hotel was a little over price...,"[The food is good, the service great.\n, Very ...",[Food was excellent with a wide range of choic...,"[Food can be a little bit overpriced, but is g...",None,None,None
2,staff_bestwestern_hotel_sfo.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,[The staff was friendly.],"[Staff is generally friendly, helpful, and eag...","[The staff is friendly and knowledgeable.\n, T...",[The staff was very helpful and gave a warm we...,"[Staff are helpful and friendly.\n, Staff are ...",None,None
3,bathroom_bestwestern_hotel_sfo.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,[People were most impressesd with the bathroom...,[Bathrooms are mostly adequate sized and clean...,"[Bathrooms are satisfactory. \n, The rooms are...",[The rooms were not large but were clean and v...,"[Rooms are not big, but clean and with comfort...",None,None
4,voice_garmin_nuvi_255W_gps.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,"[The voice is a bit robotic.\n, The voice is v...",[The voice is very clear and loud.],"[The voices sound robotic.\n, TTS mode is the ...",[255W garmin gps has more than 750 voices but ...,"[Voice is clear and sweet.\n, Voice commands a...",None,None
5,display_garmin_nuvi_255W_gps.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,"[Display is wide and clear.\n, Display can eas...","[The display is bright, clear, and smooth.\n, ...","[Display is wide, laid out nice, and is very c...","[The new screen is large, clear, and bright.\n...","[The wide display of the 225 is very good.\n, ...",None,None
6,parking_bestwestern_hotel_sfo.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,"[Parking is expensive.\n, Best to get free par...",[The expensive price of parking is unacceptabl...,[While the Best Western in San Francisco offer...,"[Parking is very expensive.\n, However, valet ...",None,None,None
7,size_asus_netbook_1005ha.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,[In spite of being small it is still comfortab...,"[The keyboard is a decent size, a bit smaller ...",[The size is great and allows for excellent po...,"[Size is small and manageable.\n, Perfect size...","[lightweight and portable, the smaller sized k...",None,None
8,staff_swissotel_chicago.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,[The staff was very cooperative and courteous.],"[Staff are kind and helpful.\n, They are both ...","[Staff was professional.\n, Front desk staff c...","[The service is friendly and well trained.\n, ...","[Some staff members can be unprofessional.\n, ...",None,None
9,free_bestwestern_hotel_sfo.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,[Evening wine reception and free coffee in the...,[Hotel offers free wine in the evening and cof...,[There is free WiFi internet access available ...,"[Free parking provided unless using valet.\n, ...",[Evening wine reception and free coffee in the...,None,None


In [20]:
dfObj.to_csv("train_test_data.csv")

In [21]:
#Encoding the sentences

from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances


In [22]:
embedding_word_list = load_encoder_dictionary()
kmeans = KMeans(init='k-means++', n_clusters=2, n_init=10)
for index, rows in dfObj.iterrows():
    if index > 2:
        break
    demoFile = open(rows["location"], "rt", encoding="utf8", errors='ignore')
    text = demoFile.read()
    sentences = sentence_spliter(text)
    #pp_Sentences  = sentence_preprocessor(sentences, embedding_word_list)
    embedding_clusters = []
    sentence_clusters  = {}
    for i,s in enumerate(sentences):
        vector = encoder.encode([s], verbose=False)
        embedding_clusters.append(vector[:,:])
        sentence_clusters[i] = s
    embedding_clusters = np.array(embedding_clusters)
    n,m,k = embedding_clusters.shape
    embedding_clusters_asa = embedding_clusters.reshape(n * m, k)
    print(embedding_clusters.shape)
    print(embedding_clusters_asa.shape)
    kmeans_fit = kmeans.fit(embedding_clusters_asa)
    label_set = list(set(kmeans_fit.labels_))
    centroids = kmeans_fit.cluster_centers_
    centroidCount = 0
    predicted_summary_list = []
    print("-------------------------------------------------------------------------Summary-Data")
    for label in label_set:
        clusterSet = embedding_clusters_asa[kmeans_fit.labels_ == label]
        euclidianDist =  distance.cdist(centroids[centroidCount:centroidCount+1,:],clusterSet,'euclidean')
        distanceArray = (np.argsort(euclidianDist)).tolist()
        print(sentence_clusters[distanceArray[0][0]])
        predicted_summary_list.append(sentence_clusters[distanceArray[0][0]])
        centroidCount= centroidCount + 1
    rows["pd1"] = predicted_summary_list
    
#print(len(sentence_list))

(78, 1, 4800)
(78, 4800)
-------------------------------------------------------------------------Summary-Data
I am loving the whole concept and read almost a whole book last night with no eye strain at all .
'Easy, on, the, eyes' reading with adjustable fonts and 16 shades of gray capability using digital ink :
 I noticed this almost immediately but thought it was due to my poor eyesight .
(109, 1, 4800)
(109, 4800)
-------------------------------------------------------------------------Summary-Data
95 if you want hot food as well .
,  Great location  ,  Very good selection of food for breakfast buffet .
(312, 1, 4800)
(312, 4800)
-------------------------------------------------------------------------Summary-Data
There was no mention from hotel staff .
The restaurant Pescatore was excellent, friendly staff, and enjoyable food .


In [ ]:
dfObj.head(20)

In [ ]:
#save the output to csv
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
fileName = "predicted_data_4800_dimensio_" + timestr + ".csv"
dfObj.to_csv(fileName)

In [ ]:
print("Predicted     \n: {}".format(dfObj.iloc[0][7]))
print("Ground Truth 1\n: {}".format(dfObj.iloc[0][2]))
print("Ground Truth 2\n: {}".format(dfObj.iloc[0][3]))
print("Ground Truth 3\n: {}".format(dfObj.iloc[0][4]))
print("Ground Truth 4\n: {}".format(dfObj.iloc[0][5]))
print("Ground Truth 5\n: {}".format(dfObj.iloc[0][6]))

In [ ]:
Converts a multi-dimenson array in to 2D array, that would be easier for visualization.
embedding_clusters = np.array(embedding_clusters)
n,m,k = embedding_clusters.shape
embedding_clusters_asa = embedding_clusters.reshape(n * m, k)
print(embedding_clusters.shape)
print(embedding_clusters_asa.shape)

In [ ]:
Converting the 4800 dimension to single dimension
tsne_model_en_2d = TSNE(n_iter=5000)
embeddings_en_2d = np.array(tsne_model_en_2d.fit_transform(embedding_clusters_asa))
print(embeddings_en_2d.shape)

In [ ]:

#print(label_set)

In [ ]:
#finding the euclidian distance, between points


#print(kmeans_fit.labels_)

    
